
# GHC 2025 – Scaling Data Enrichment using LLMs (Workshop)
**By Muskaan Goyal & Tejashri Naikar**

Welcome! In this notebook, we will build a data enrichment pipeline using a real-estate dataset to convert basic numerical housing features into semantic signals (e.g., coffee access, food corridor, transit convenience, neighborhood vibe). We will then rebuild retrieval on top of those enriched signals to show how search suddenly becomes aligned with the way humans actually ask questions.


**Our Goal**: To demonstrate how to use a Large Language Model (LLM) to enrich data at scale.


**Learning Objectives**: By the end of this workshop, you will be able to:
1. Build a baseline vector retrieval system and diagnose why it fails on human questions.
2. Use prompt engineering patterns (Zero-Shot, Few-Shot, JSON Schema, CoT) to produce structured enrichment.
3. Make model selection decisions based on quality, latency, and cost tradeoffs.
4. Rebuild retrieval over enriched semantic signals and measure the improvement.



---





## 0. Install dependencies

We will install all the libraries needed for the full pipeline:

> **kagglehub** - to download our dataset directly from Kaggle (no Kaggle API key required)

> **sentence-transformers** — to generate vector embeddings from text

> **faiss-cpu** — to build a high-performance vector search index

> **pandas** — to manipulate tabular housing data

> **tqdm** — to visualize progress when enriching rows



In [ ]:
!pip -q install kagglehub==0.3.5 sentence-transformers==3.0.1 faiss-cpu==1.8.0.post1 pandas==2.2.2 tqdm==4.67.0


## 1. Configure OpenRouter (LLM) access
Paste your **OpenRouter API key** when prompted.  
You can create one at https://openrouter.ai/ (free models available).


In [46]:

import getpass, os

OPENROUTER_API_KEY = getpass.getpass("Paste your OpenRouter API key (input is hidden): ").strip()
os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY

Paste your OpenRouter API key (input is hidden): ··········


In [47]:
OPENROUTER_API_URL = "https://openrouter.ai/api/v1/chat/completions"

DEFAULT_MODEL = "google/gemma-3-4b-it"
MODEL_OPTIONS = [
    "mistralai/mistral-small-3-instruct",
    "meta-llama/llama-3.2-3b-instruct",
    "google/gemma-3-4b-it"
]
print("Configured OpenRouter. Default model:", DEFAULT_MODEL)


Configured OpenRouter. Default model: google/gemma-3-4b-it



## 2. Data Ingestion

We will use the [King County (WA) Home Sales dataset](https://www.kaggle.com/datasets/andykrause/kingcountysales). This dataset contains more than 20 years of residential home sales for King County, Washington (Seattle area) with columns about sale price, property size, and other home characteristics.



In [249]:
import kagglehub, os, pandas as pd

path = kagglehub.dataset_download("andykrause/kingcountysales")
print("Downloaded to:", path)

# The dataset provides a CSV file named 'kc_house_data.csv' (or similar)
csv_path = None
for root, _, files in os.walk(path):
    for f in files:
        if f.lower().endswith(".csv"):
            csv_path = os.path.join(root, f)
            break
    if csv_path:
        break

assert csv_path, "CSV file not found in kagglehub dataset folder."
df = pd.read_csv(csv_path)
print("Rows:", len(df), "Columns:", df.columns.tolist())
df.head()


Downloaded to: /root/.cache/kagglehub/datasets/andykrause/kingcountysales/versions/8
Rows: 560219 Columns: ['Unnamed: 0', 'sale_id', 'pinx', 'sale_date', 'sale_price', 'sale_nbr', 'sale_warning', 'join_status', 'join_year', 'latitude', 'longitude', 'area', 'city', 'zoning', 'subdivision', 'present_use', 'land_val', 'imp_val', 'year_built', 'year_reno', 'sqft_lot', 'sqft', 'sqft_1', 'sqft_fbsmt', 'grade', 'fbsmt_grade', 'condition', 'stories', 'beds', 'bath_full', 'bath_3qtr', 'bath_half', 'garb_sqft', 'gara_sqft', 'wfnt', 'golf', 'greenbelt', 'noise_traffic', 'view_rainier', 'view_olympics', 'view_cascades', 'view_territorial', 'view_skyline', 'view_sound', 'view_lakewash', 'view_lakesamm', 'view_otherwater', 'view_other', 'submarket']


,Unnamed: 0,sale_id,pinx,sale_date,sale_price,sale_nbr,sale_warning,join_status,join_year,latitude,...,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other,submarket
0,1,1999..144,..2734100475,1999-01-05,150000,1,,demo,1999,47.544359,...,0,0,0,0,0,0,0,0,0,E
1,2,1999..258,..1535200725,1999-01-05,235000,1,,demo,1999,47.421247,...,0,0,2,0,0,0,0,0,0,H
2,3,1999..331,..6028000255,1999-01-04,293000,1,,demo,1999,47.572103,...,0,0,0,0,0,0,0,0,0,R
3,4,1999..660,..6145600690,1999-01-08,164000,1,,demo,1999,47.703824,...,0,0,0,0,0,0,0,0,0,B
4,5,1999..775,..1939800005,1999-01-07,270000,1,,demo,1999,47.764482,...,0,0,0,0,0,0,0,0,0,Q



## 3. Data Cleaning
For performance, we will restrict our dataset size to 500 homes in seattle.


In [252]:
import numpy as np

df_seattle = df[df['city'] == 'SEATTLE']
df_sample = df_seattle.dropna().reset_index(drop=True)
df_sample = df_sample[:500]
print("Sample rows:", len(df_sample))
df_sample.head()

Sample rows: 500


,Unnamed: 0,sale_id,pinx,sale_date,sale_price,sale_nbr,sale_warning,join_status,join_year,latitude,...,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other,submarket
0,1,1999..144,..2734100475,1999-01-05,150000,1,,demo,1999,47.544359,...,0,0,0,0,0,0,0,0,0,E
1,4,1999..660,..6145600690,1999-01-08,164000,1,,demo,1999,47.703824,...,0,0,0,0,0,0,0,0,0,B
2,6,1999..1155,..3438501120,1999-01-19,175000,1,,demo,1999,47.549525,...,0,0,0,0,0,0,0,0,0,F
3,9,1999..1354,..3336001680,1999-01-14,160000,1,,demo,1999,47.526324,...,0,0,0,0,0,0,0,0,0,E
4,10,1999..1424,..7857000705,1999-01-19,135000,1,,demo,1999,47.550687,...,0,0,0,0,0,0,0,0,0,E


We will rename some of the columns to improve readability.

In [253]:
df_sample = df_sample.rename(columns={'bath_full': 'bathrooms', 'beds': 'bedrooms'})
print("Columns renamed successfully.")
df_sample.head()

Columns renamed successfully.


,Unnamed: 0,sale_id,pinx,sale_date,sale_price,sale_nbr,sale_warning,join_status,join_year,latitude,...,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other,submarket
0,1,1999..144,..2734100475,1999-01-05,150000,1,,demo,1999,47.544359,...,0,0,0,0,0,0,0,0,0,E
1,4,1999..660,..6145600690,1999-01-08,164000,1,,demo,1999,47.703824,...,0,0,0,0,0,0,0,0,0,B
2,6,1999..1155,..3438501120,1999-01-19,175000,1,,demo,1999,47.549525,...,0,0,0,0,0,0,0,0,0,F
3,9,1999..1354,..3336001680,1999-01-14,160000,1,,demo,1999,47.526324,...,0,0,0,0,0,0,0,0,0,E
4,10,1999..1424,..7857000705,1999-01-19,135000,1,,demo,1999,47.550687,...,0,0,0,0,0,0,0,0,0,E


## 4. Baseline Retrieval (Data Indexing & Retrieval)

In this step, we build a **baseline vector search**. We convert each listing into a very simple text string (just raw key values like beds, baths, sqft, year built, etc.), embed that text using a small model, and store those vectors in FAISS (Facebook AI Similarity Search). FAISS is a library that lets us build vector indexes and perform fast nearest-neighbor lookups over embeddings.

This is our control case. We have no semantic enrichment yet. We will run natural language queries against this naive index first, so we can clearly see what raw signals can and cannot retrieve before we introduce richer representations.


### 4.1 Code

In [317]:

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Choose a small, fast embedding model
EMBED_MODEL_NAME = "all-MiniLM-L6-v2"
emb_model = SentenceTransformer(EMBED_MODEL_NAME)

RAW_COLUMNS = ['sqft', "bedrooms", "bathrooms", "city", "year_built", 'view_olympics', 'view_cascades', 'view_territorial', 'view_skyline', 'view_sound', 'view_lakewash', 'view_lakesamm']
available_cols = [c for c in RAW_COLUMNS if c in df_sample.columns]
assert available_cols, f"None of the expected raw columns found. Have: {df_sample.columns.tolist()}"

def row_to_raw_text(r):
    parts = []
    for col in available_cols:
        val = r[col]
        parts.append(f"{col}={val}")
    return "; ".join(parts)

df_sample["raw_text"] = df_sample.apply(row_to_raw_text, axis=1)

# Embed and index raw text
raw_embeddings = np.array(
    emb_model.encode(
        df_sample["raw_text"].tolist(),
        convert_to_numpy=True,
        show_progress_bar=True
    ),
    dtype="float32"
)
index_raw = faiss.IndexFlatL2(raw_embeddings.shape[1])
index_raw.add(raw_embeddings)


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [314]:
def search_raw(query, k=5, index=index_raw):
    q = emb_model.encode([query], convert_to_numpy=True).astype("float32")
    D, I = index.search(q, k)
    return I[0], D[0]

### 4.2 Examples

#### Example 1 (Basic Query)

In this example, we will start with a basic query for retrieving 4 Bedroom 2 Bathroom Houses in Seattles


In [318]:
USER_QUERY = "Find me 4 bedroom 2 bathroom houses in Seattle"

top_idx, top_dist = search_raw(USER_QUERY, k=5)
df_sample.iloc[top_idx][available_cols + ["raw_text"]]

,sqft,bedrooms,bathrooms,city,year_built,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,raw_text
411,2780,4,1,SEATTLE,1925,0,0,0,0,0,0,0,sqft=2780; bedrooms=4; bathrooms=1; city=SEATT...
258,1264,3,1,SEATTLE,1900,0,0,0,0,0,0,0,sqft=1264; bedrooms=3; bathrooms=1; city=SEATT...
50,2270,4,2,SEATTLE,1922,0,0,0,0,0,1,0,sqft=2270; bedrooms=4; bathrooms=2; city=SEATT...
145,1260,4,1,SEATTLE,1910,0,0,0,0,0,0,0,sqft=1260; bedrooms=4; bathrooms=1; city=SEATT...
283,2190,5,2,SEATTLE,1938,0,0,0,0,0,0,0,sqft=2190; bedrooms=5; bathrooms=2; city=SEATT...


After running this search, we can observe that our semantic search successfully retrieved houses partially matching our criteria, with most (if not all depending on the sample) results being exact 4-bedroom matches in Seattle. Notably, these properties span across different years, demonstrating that our search effectively finds relevant results regardless of the property's age.

But this is a very simple query that could have been answered through filtering. So let's move on to more complex queries.



#### Example 2 (Slightly Complex Query)

Now, let's try a more complex search by adding view preferences to our query. While our previous search focused on basic attributes like bedrooms and bathrooms, this time we'll ask for homes with beautiful views. This will help test our semantic search's ability to interpret and match property features.

In [319]:
USER_QUERY = "Find me 4 bedroom 2 bathroom homes with views of lakes in Seattle"

top_idx, top_dist = search_raw(USER_QUERY, k=5)
df_sample.iloc[top_idx][available_cols + ["raw_text"]]

,sqft,bedrooms,bathrooms,city,year_built,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,raw_text
283,2190,5,2,SEATTLE,1938,0,0,0,0,0,0,0,sqft=2190; bedrooms=5; bathrooms=2; city=SEATT...
416,1400,4,1,SEATTLE,1926,0,0,2,0,0,0,0,sqft=1400; bedrooms=4; bathrooms=1; city=SEATT...
235,2190,2,1,SEATTLE,1926,0,0,0,0,0,0,0,sqft=2190; bedrooms=2; bathrooms=1; city=SEATT...
215,850,3,1,SEATTLE,1962,0,0,0,0,0,0,0,sqft=850; bedrooms=3; bathrooms=1; city=SEATTL...
248,850,3,1,SEATTLE,1962,0,0,0,0,0,0,0,sqft=850; bedrooms=3; bathrooms=1; city=SEATTL...


After running this more complex query, we can see an interesting limitation of our basic semantic search. Despite having specific columns in our dataset for different types of views (Olympics, Cascades, Sound, etc.), our search returned homes with no views (all 0s in view columns) even though the query explicitly asked for "beautiful views".

This perfectly illustrates **the retrieval gap**, where our search system fails to make meaningful connections between structured data attributes and natural language queries, even when the relevant information exists in our dataset.


#### Example 3 (Most Realistic Query)

Going beyond just property features, now let's test how our semantic search handles neighborhood amenities. We'll now search for homes that combine specific property criteria (4 bedrooms, 2 bathroom) with location-based conveniences like restaurant accessibility, transit options, and coffee shops.

These factors often play a higher role in influencing real-world housing decisions.

In [320]:
USER_QUERY = "Find me 4 bedroom 2 bathroom homes in seattle with close proximity to restaurants and transit"

top_idx, top_dist = search_raw(USER_QUERY, k=5)
df_sample.iloc[top_idx][available_cols + ["raw_text"]]

,sqft,bedrooms,bathrooms,city,year_built,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,raw_text
411,2780,4,1,SEATTLE,1925,0,0,0,0,0,0,0,sqft=2780; bedrooms=4; bathrooms=1; city=SEATT...
258,1264,3,1,SEATTLE,1900,0,0,0,0,0,0,0,sqft=1264; bedrooms=3; bathrooms=1; city=SEATT...
191,1430,4,1,SEATTLE,1900,0,0,0,0,0,0,0,sqft=1430; bedrooms=4; bathrooms=1; city=SEATT...
145,1260,4,1,SEATTLE,1910,0,0,0,0,0,0,0,sqft=1260; bedrooms=4; bathrooms=1; city=SEATT...
215,850,3,1,SEATTLE,1962,0,0,0,0,0,0,0,sqft=850; bedrooms=3; bathrooms=1; city=SEATTL...


Here the issue appears similar to our previous views query where our search returned homes matching only the basic 2-bedroom, 1-bathroom criteria while ignoring the additional requirements.

However, the underlying cause is different. In the previous example, we had the relevant data (view columns) but our retrieval system couldn't make the semantic connection. Here, the raw schema simply lacks attributes like coffee access, food density, or transit convenience. This issue happens because **vector search can't retrieve what isn't represented**.



---

### **[Break]**

*Please take a pause here for now. We will jump back into the remaining sections after a mini-lecture.*

---

## 5. LLM Based Data Enrichment

Our previous examples highlighted two different retrieval gaps, each leading to distinct opportunities for LLM-based enrichment:

1. Extraction and Summarization
 - Where data (views) existed but semantic connection failed
 - LLMs can help extract and summarize existing structured data into more meaningful representations

2. Augmentation
 - Where desired attributes (like neighborhood amenities) were completely missing
 - LLMs can augment our data by generating new attributes based on available information

Let's now explore how we can use LLMs to address both these opportunities, enriching our raw property data with semantic attributes that better capture these missing dimensions.


In [261]:
import json, time, hashlib, requests, os
from typing import Dict, Any

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY", "").strip()
assert OPENROUTER_API_KEY, "OpenRouter API key missing. Please re-run Step #1."


def call_llm_json(prompt: str, model: str = DEFAULT_MODEL, max_retries: int = 2, debug_usage=False) -> Dict[str, Any]:
    """Call OpenRouter chat completions expecting JSON in response. We parse last block as JSON."""
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "HTTP-Referer": "https://colab.research.google.com",
        "X-Title": "GHC Data Enrichment Workshop"
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a data enrichment model. Output ONLY valid JSON matching the provided schema."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.2
    }
    for attempt in range(1, max_retries+1):
        r = requests.post(OPENROUTER_API_URL, headers=headers, json=payload, timeout=60)
        if r.status_code == 200:
            data = r.json()
            if debug_usage:
              print("Usage:", data["usage"])
            try:
                content = data["choices"][0]["message"]["content"]
                # Attempt to locate JSON block
                start = content.find("{")
                end = content.rfind("}")
                if start != -1 and end != -1:
                    return json.loads(content[start:end+1])
            except Exception:
                pass
        time.sleep(1.2 * attempt)
    raise RuntimeError(f"Failed to parse JSON from model after {max_retries} attempts. Last status={getattr(r,'status_code',None)}")


### 5.1 Prompt Engineering for Summarization Use-Case

Now that we have theoretically learnt different types of prompting techniques, lets apply them to improve how we extract and summarize property views data, addressing our earlier semantic connection gap.

We will experiment these techniques first on two rows to evaluate accuracy before applying it on all the dataset.

In [262]:
# Let's first run this enrichment for one row
row_without_view = df_sample.iloc[0].to_dict()
print(df_sample.iloc[0])

Unnamed: 0                                                          1
sale_id                                                     1999..144
pinx                                                     ..2734100475
sale_date                                                  1999-01-05
sale_price                                                     150000
sale_nbr                                                            1
sale_warning                                                         
join_status                                                      demo
join_year                                                        1999
latitude                                                    47.544359
longitude                                                 -122.320786
area                                                               78
city                                                          SEATTLE
zoning                                                        SF 5000
subdivision         

In [266]:
# Let's first run this enrichment for one row
row_with_view = df_sample.iloc[40].to_dict()
print(df_sample.iloc[40])

Unnamed: 0                                                         87
sale_id                                                   1999..12379
pinx                                                     ..2824600195
sale_date                                                  1999-04-14
sale_price                                                     318000
sale_nbr                                                            1
sale_warning                                                         
join_status                                                      demo
join_year                                                        1999
latitude                                                    47.575418
longitude                                                   -122.3778
area                                                               48
city                                                          SEATTLE
zoning                                                        SF 5000
subdivision         

In [272]:
addn_row_with_view = df_sample.iloc[68].to_dict()
print(df_sample.iloc[30])

Unnamed: 0                                                         70
sale_id                                                   1999..10420
pinx                                                     ..9530101040
sale_date                                                  1999-04-08
sale_price                                                     226500
sale_nbr                                                            1
sale_warning                                                         
join_status                                                      demo
join_year                                                        1999
latitude                                                     47.66522
longitude                                                  -122.36005
area                                                               42
city                                                          SEATTLE
zoning                                                        SF 5000
subdivision         

#### 5.1.1 Zero Shot Prompting

Let's first try zero shot prompting

In [268]:
def make_zero_shot_prompt(row: dict) -> str:
    return f"""You are enriching a housing row with semantic attributes people care about. You have access to different columns related to views. View with value 1 means that the home has that veiw else no view. Summarize the views available from this home. If no views are available, return no prime views"""


In [273]:
print("-----")
print("LLM Enrichment for row without any view")
print(call_llm_json(make_zero_shot_prompt(row_without_view), debug_usage=True))
print("-----")
print("LLM Enrichment for row with good view")
print(call_llm_json(make_zero_shot_prompt(row_with_view), debug_usage=True))
print("-----")
print("LLM Enrichment for row 2 with good view")
print(call_llm_json(make_zero_shot_prompt(addn_row_with_view), debug_usage=True))
print("-----")

-----
LLM Enrichment for row without any view
Usage: {'prompt_tokens': 85, 'completion_tokens': 21, 'total_tokens': 106, 'prompt_tokens_details': None}
{'prime_views': 'Garden, Street'}
-----
LLM Enrichment for row with good view
Usage: {'prompt_tokens': 85, 'completion_tokens': 21, 'total_tokens': 106, 'prompt_tokens_details': None}
{'prime_views': 'Garden, Street'}
-----
LLM Enrichment for row 2 with good view
Usage: {'prompt_tokens': 85, 'completion_tokens': 21, 'total_tokens': 106, 'prompt_tokens_details': None}
{'prime_views': 'Garden, Street'}
-----


This row highlights a critical challenge with LLM-based data enrichment: hallucination. While our model returned {'prime_views': 'Garden, Street'}, we can see that not only are all actual view columns (Olympics, Cascades, Sound, etc.) marked as 0, but more importantly, our data schema doesn't even contain columns for garden or street views.

The LLM fabricated these attributes by drawing from its general knowledge about houses rather than strictly working with the available data. This demonstrates why careful prompt engineering is crucial to constrain LLMs to only extract and summarize existing attributes, not invent new ones.

In [270]:
### Let's try to fine tune our prompt
def make_zero_shot_prompt_opt(row: dict) -> str:
    return f"""
      Given a house with the following view ratings (0 means no view, values > 0 indicate presence of view):
      - Mount Rainier: {row['view_rainier']}
      - Olympics: {row['view_olympics']}
      - Cascades: {row['view_cascades']}
      - Territorial: {row['view_territorial']}
      - Skyline: {row['view_skyline']}
      - Sound: {row['view_sound']}
      - Lake Washington: {row['view_lakewash']}
      - Lake Sammamish: {row['view_lakesamm']}
      - Other Water: {row['view_otherwater']}

      Write a brief, natural description of the views this house offers. Only mention views that have values greater than 0.
      Make the description flow naturally, grouping similar views together (e.g., mountain views, water views, city views).
      If no views are present (all 0s), return "This house has no notable views."
    """


In [274]:
print("-----")
print("LLM Enrichment for row without any view")
print(call_llm_json(make_zero_shot_prompt_opt(row_without_view), debug_usage=True))
print("-----")
print("LLM Enrichment for row with good view")
print(call_llm_json(make_zero_shot_prompt_opt(row_with_view), debug_usage=True))
print("-----")
print("LLM Enrichment for row 2 with good view")
print(call_llm_json(make_zero_shot_prompt_opt(addn_row_with_view), debug_usage=True))
print("-----")

-----
LLM Enrichment for row without any view
Usage: {'prompt_tokens': 193, 'completion_tokens': 22, 'total_tokens': 215, 'prompt_tokens_details': None}
{'description': 'This house offers no notable views.'}
-----
LLM Enrichment for row with good view
Usage: {'prompt_tokens': 193, 'completion_tokens': 42, 'total_tokens': 235, 'prompt_tokens_details': None}
{'description': 'This house offers a glimpse of the Cascades and a pleasant view of the Sound. It also has a small view of Lake Washington.'}
-----
LLM Enrichment for row 2 with good view
Usage: {'prompt_tokens': 193, 'completion_tokens': 35, 'total_tokens': 228, 'prompt_tokens_details': None}
{'description': 'This house offers a serene setting with no prominent views of mountains, water bodies, or cityscapes.'}
-----


[Insight] *While our improved prompt effectively prevented hallucination, it came at a cost - nearly doubling the total tokens from 106 to 245. However, this trade-off between token usage and accuracy is often worthwhile when data quality is crucial.*

#### 5.1.2 One Shot Prompting

In [156]:
def make_one_shot_prompt(row: dict) -> str:
    return f"""
      Given a house with view ratings (0 means no view, values > 0 indicate presence of view), write a natural description of the views. Only mention views with values greater than 0.

      Example Input:
      - Mount Rainier: 0
      - Olympics: 3
      - Cascades: 2
      - Territorial: 4
      - Skyline: 3
      - Sound: 0
      - Lake Washington: 0
      - Lake Sammamish: 0
      - Other Water: 0

      Example Output:
      This home features stunning mountain views of the Olympics and Cascades. It also offers expansive territorial and city skyline views.

      Now describe the views for this house:
      - Mount Rainier: {row['view_rainier']}
      - Olympics: {row['view_olympics']}
      - Cascades: {row['view_cascades']}
      - Territorial: {row['view_territorial']}
      - Skyline: {row['view_skyline']}
      - Sound: {row['view_sound']}
      - Lake Washington: {row['view_lakewash']}
      - Lake Sammamish: {row['view_lakesamm']}
      - Other Water: {row['view_otherwater']}
    """

In [157]:
print("-----")
print("LLM Enrichment for row without any view")
print(call_llm_json(make_one_shot_prompt(row_without_view), debug_usage=True))
print("-----")
print("LLM Enrichment for row with good view")
print(call_llm_json(make_one_shot_prompt(row_with_view), debug_usage=True))
print("-----")
print("LLM Enrichment for row 2 with good view")
print(call_llm_json(make_one_shot_prompt(addn_row_with_view), debug_usage=True))
print("-----")

-----
LLM Enrichment for row without any view
Usage: {'prompt_tokens': 248, 'completion_tokens': 22, 'total_tokens': 270, 'prompt_tokens_details': None}
{'description': 'This home has no notable views.'}
-----
LLM Enrichment for row with good view
Usage: {'prompt_tokens': 248, 'completion_tokens': 45, 'total_tokens': 293, 'prompt_tokens_details': None}
{'description': 'This home boasts spectacular views of the Olympics, Cascades, Territorial, Skyline, and Lake Washington. It also offers a pleasant view of the Sound.'}
-----
LLM Enrichment for row 2 with good view
Usage: {'prompt_tokens': 248, 'completion_tokens': 25, 'total_tokens': 273, 'prompt_tokens_details': None}
{'description': 'This home offers a pleasant view of the Sound.'}
-----


**[Insight]** *We notice that tokens size increase when we do one-shot prompting while the results remain similar. This example shows us that adding examples is not always the best prompt possible.*

#### 5.1.3 Chain of Thought Prompting

In [169]:
def make_chain_thought_prompt(row: dict) -> str:
    view_data = [f"{k.replace('view_', '')}: {v}" for k, v in row.items() if k.startswith('view_')]
    return f"""
    Let's analyze the views of this house through a step-by-step process internally:

    Step 1: Here is the EXACT view data (values of 0 mean NO view):
    {', '.join(view_data)}

    Step 2: STRICT RULES for view inclusion:
    - A view must have a value GREATER THAN 0 to be mentioned
    - Example: if rainier: 0, DO NOT mention Rainier views
    - Example: if sound: 1, DO mention Sound views
    - DO NOT mention any views not listed above
    - DO NOT infer or add additional views

    Step 3: Using ONLY views with values > 0, create a natural description.

    Return ONLY the final natural description.
    If ALL views have value 0, you MUST return exactly "This house has no notable views."

    FINAL CHECK: Before returning, verify each view mentioned has a value > 0 in the data above.
    """


In [170]:
print("-----")
print("LLM Enrichment for row without any view")
print(call_llm_json(make_chain_thought_prompt(row_without_view), debug_usage=True))
print("-----")
print("LLM Enrichment for row with good view")
print(call_llm_json(make_chain_thought_prompt(row_with_view), debug_usage=True))
print("-----")
print("LLM Enrichment for row 2 with good view")
print(call_llm_json(make_chain_thought_prompt(addn_row_with_view), debug_usage=True))
print("-----")

-----
LLM Enrichment for row without any view
Usage: {'prompt_tokens': 282, 'completion_tokens': 22, 'total_tokens': 304, 'prompt_tokens_details': None}
{'description': 'This house has no notable views.'}
-----
LLM Enrichment for row with good view
Usage: {'prompt_tokens': 282, 'completion_tokens': 48, 'total_tokens': 330, 'prompt_tokens_details': None}
{'description': 'This house offers views of the Olympics, Cascades, Territorial, Skyline, and Sound. It also has notable views of the Lakewash and the Olympic Mountains.'}
-----
LLM Enrichment for row 2 with good view
Usage: {'prompt_tokens': 282, 'completion_tokens': 22, 'total_tokens': 304, 'prompt_tokens_details': None}
{'description': 'This house has a Sound view.'}
-----


**[Insight]** *Note that the above response does not show a lot of improvement so we should still prefer zero-shot prompting for this use case*.


#### 5.1.4 PlayGround for COT

We now want to consider different levels of view, so try to write a prompt using COT technique for our view description. We have provided an example at the end but please first try to write one yourself.

In [179]:
def make_chain_thought_prompt_with_quality_of_view(row: dict) -> str:
    view_data = [f"{k.replace('view_', '')}: {v}" for k, v in row.items() if k.startswith('view_')]
    return f"""
    ADD Instructions
    """

# Test Enrichments
# print("-----")
# print("LLM Enrichment for row without any view")
# print(call_llm_json(make_chain_thought_prompt_with_quality_of_view(row_without_view), debug_usage=True))
# print("-----")
# print("LLM Enrichment for row with good view")
# print(call_llm_json(make_chain_thought_prompt_with_quality_of_view(row_with_view), debug_usage=True))
# print("-----")
# print("LLM Enrichment for row 2 with good view")
# print(call_llm_json(make_chain_thought_prompt_with_quality_of_view(addn_row_with_view), debug_usage=True))
# print("-----")

Here is a sample example:

In [180]:
def make_chain_thought_prompt_with_quality_of_view(row: dict) -> str:
    view_data = [f"{k.replace('view_', '')}: {v}" for k, v in row.items() if k.startswith('view_')]
    return f"""
    Let's analyze the views of this house through a step-by-step process internally:

    Step 1: Here is the EXACT view data (values of 0 mean NO view):
    {', '.join(view_data)}

    Step 2: STRICT RULES for view inclusion and quality assessment:
    - A view must have a value GREATER THAN 0 to be mentioned
    - View values indicate quality:
        * 4: exceptional/spectacular
        * 3: excellent
        * 2: good
        * 1: partial/limited

    Step 3: Using ONLY views with values > 0:
    - Group views by type (mountain, water, urban)
    - Consider the quality based on the values
    - Create a description that reflects both presence and quality

    Return ONLY the final natural description.
    If ALL views have value 0, return exactly "This house has no notable views."

    FINAL CHECK: Verify each view mentioned has a value > 0 in the data above.
    """
print("-----")
print("LLM Enrichment for row without any view")
print(call_llm_json(make_chain_thought_prompt_with_quality_of_view(row_without_view), debug_usage=True))
print("-----")
print("LLM Enrichment for row with good view")
print(call_llm_json(make_chain_thought_prompt_with_quality_of_view(row_with_view), debug_usage=True))
print("-----")
print("LLM Enrichment for row 2 with good view")
print(call_llm_json(make_chain_thought_prompt_with_quality_of_view(addn_row_with_view), debug_usage=True))
print("-----")


-----
LLM Enrichment for row without any view
Usage: {'prompt_tokens': 297, 'completion_tokens': 65, 'total_tokens': 362, 'prompt_tokens_details': None}
{'description': 'This house offers a partial view of the Olympic Mountains.', 'views': [{'type': 'mountain', 'name': 'olympics', 'quality': 2}]}
-----
LLM Enrichment for row with good view
Usage: {'prompt_tokens': 297, 'completion_tokens': 75, 'total_tokens': 372, 'prompt_tokens_details': None}
{'description': 'This house offers spectacular views of the Olympics, Cascades, Territorial, and Skyline. It also has excellent views of the Olympics and Cascades, and a good view of the Sound. The house also has a partial view of the lakes and other water features, and a limited view of other areas.'}
-----
LLM Enrichment for row 2 with good view
Usage: {'prompt_tokens': 297, 'completion_tokens': 63, 'total_tokens': 360, 'prompt_tokens_details': None}
{'description': 'This house offers a partial view of the Sound.', 'views': [{'type': 'water', 

### 5.2 Enriched Vector Search for Summarization Use Case

Since zero shot prompting was performing well, we will try to enrich the dataset using this prompting technique for our sampled dataset and see how the retrieval changes.

In [275]:
# Now lets apply this data enrichment for the whole sampled dataset. This code should take about ~4 minutes to run for 150 rows.

collected_dfs = []

def enrich_row_safe(row):
    try:
        return call_llm_json(make_zero_shot_prompt_opt(row))["description"]
    except Exception as e:
        print(f"Error enriching row (index: {row.name}): {e}")
        return ""

# Loop for batch processing, assuming chunks of 50 rows
for i in range(0, 3):
  start_idx = i * 50
  end_idx = (i + 1) * 50

  df_chunk = df_sample.iloc[start_idx:min(end_idx, len(df_sample))].copy()

  if df_chunk.empty:
      print(f"No more rows to process; reached end of df_sample at iteration {i}.")
      break

  df_chunk["view description"] = df_chunk.apply(enrich_row_safe, axis=1)

  # Sleep for 5 seconds after processing each chunk to avoid rate limits
  time.sleep(5)

  collected_dfs.append(df_chunk)
  print(f"Enriched {len(df_chunk)} rows in chunk {i+1}. Total enriched so far: {sum(len(d) for d in collected_dfs)} rows.")


if collected_dfs:
    enriched_df = pd.concat(collected_dfs).reset_index(drop=True)
    print(f"\nSuccessfully enriched a total of {len(enriched_df)} rows.")
else:
    enriched_df = pd.DataFrame() # Create an empty DataFrame if no data was processed
    print("\nNo rows were successfully enriched.")

Error enriching row (index: 6): 'description'
Error enriching row (index: 16): 'description'
Enriched 50 rows in chunk 1. Total enriched so far: 50 rows.
Error enriching row (index: 71): 'description'
Error enriching row (index: 73): 'description'
Error enriching row (index: 75): 'description'
Error enriching row (index: 80): 'description'
Error enriching row (index: 82): 'description'
Error enriching row (index: 88): 'description'
Error enriching row (index: 92): 'description'
Enriched 50 rows in chunk 2. Total enriched so far: 100 rows.
Error enriching row (index: 105): 'description'
Error enriching row (index: 108): 'description'
Error enriching row (index: 111): 'description'
Error enriching row (index: 120): 'description'
Error enriching row (index: 147): 'description'
Enriched 50 rows in chunk 3. Total enriched so far: 150 rows.

Successfully enriched a total of 150 rows.


In [309]:
RAW_COLUMNS = ['bedrooms', 'bathrooms', 'city', 'view description']
available_cols = [c for c in RAW_COLUMNS if c in enriched_df.columns]
assert available_cols, f"None of the expected raw columns found. Have: {enriched_df.columns.tolist()}"

def row_to_raw_text(r):
    parts = []
    for col in available_cols:
        val = r[col]
        parts.append(f"{col}={val}")
    return "; ".join(parts)

enriched_df["raw_text"] = enriched_df.apply(row_to_raw_text, axis=1)

# Embed and index raw text
raw_embeddings_enriched = np.array(
    emb_model.encode(
        enriched_df["raw_text"].tolist(),
        convert_to_numpy=True,
        show_progress_bar=True
    ),
    dtype="float32"
)
index_enriched = faiss.IndexFlatL2(raw_embeddings_enriched.shape[1])
index_enriched.add(raw_embeddings_enriched)

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [321]:
USER_QUERY = "Find me 4 bedroom 2 bathroom homes with views of lakes in Seattle"

top_idx, top_dist = search_raw(USER_QUERY, k=5, index = index_enriched)
enriched_df.iloc[top_idx][available_cols + ["raw_text"]]

,sqft,bedrooms,bathrooms,city,year_built,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,raw_text
40,2090,4,2,SEATTLE,1955,0,2,0,0,1,0,0,bedrooms=4; bathrooms=2; city=SEATTLE; view de...
50,2270,4,2,SEATTLE,1922,0,0,0,0,0,1,0,bedrooms=4; bathrooms=2; city=SEATTLE; view de...
24,2420,3,3,SEATTLE,1908,2,0,0,0,1,0,0,bedrooms=3; bathrooms=3; city=SEATTLE; view de...
35,1920,4,2,SEATTLE,1945,0,2,0,0,0,0,0,bedrooms=4; bathrooms=2; city=SEATTLE; view de...
146,2040,4,2,SEATTLE,1948,2,0,2,0,2,0,0,bedrooms=4; bathrooms=2; city=SEATTLE; view de...


**Yayyyy 🤍!**

We can finally see that we were not able to retrieve 4 bedroom 4 bathroom apartments with views of lakes in our top 2 results.

This example shows the power of LLM based enrichment for improving data retrieval.





---

[Muskaan has updated it till here]

---


We will prompt the LLM to infer **scores** (0–5) and **tags** for each home.  
**Target schema** per row:
```json
{
  "coffee_access_score": 0-5,
  "food_access_score": 0-5,
  "transit_access_score": 0-5,
  "lifestyle_tags": ["walkable","near_coffee","near_food","near_transit","quiet","downtown","suburban","young_professional_friendly"]
}
```
We'll start with **Zero-Shot**, then improve using **Few-Shot**, **JSON Schema**, and **Chain-of-Thought**.


### 5.3 Prompt Engineering for Data Augmentation Use Case

Now let's move on to our use case of data augmentation where we will use neighborhood information for a house to augment data for coffee access and transit access.

In [326]:
def make_neighborhood_prompt(row: dict) -> str:
    return f"""
    For a house at {row['latitude']}, {row['longitude']} in {row['city']}, {row['subdivision']},
    provide a BRIEF assessment of key transit and amenity access.

    Example 1:
    Input: Capitol Hill, Seattle
    Output: Transit: 2 min walk to light rail, multiple bus lines
    Amenities: 5 min walk to coffee, restaurants, groceries
    Commute: 10 min to downtown

    Example 2:
    Input: Maple Valley, WA
    Output: Transit: Limited - hourly bus service, no rail
    Amenities: 15 min drive to shops and restaurants
    Commute: 45 min to downtown

    Return 3 lines ONLY:
    Line 1: Transit: [distance to nearest transit options]
    Line 2: Amenities: [distance to coffee, restaurants, groceries]
    Line 3: Commute: [typical time to downtown]
    Use specific minutes/distances.
    """

Can you guess the the prompt engineering technique used here?

In [ ]:
print("LLM Enrichment for " + row_without_view["subdivision"])
print(call_llm_json(make_neighborhood_prompt(row_without_view), debug_usage=True))

In the above example, we can see that LLM is able to correctly extract neighborhood information for "GEORGETOWN" neighborhood in Seattle.

**Playground**

In [ ]:
# Feel free to play around here with the prompt and try to augment data that is helpful to make housing decisions.

### 5.4 Enriched Vector Search For Data Augmentation Use Case

Note that the following cell should take about ~8 minutes to run.

In [338]:
collected_dfs = []

def enrich_row_safe(row):
    try:
        # Initialize default empty values without using row.apply()
        transit, amenities, commute = "", "", ""

        data = call_llm_json(make_neighborhood_prompt(row))
        if isinstance(data, dict):
            transit = data.get("transit", "")
            amenities = data.get("amenities", "")
            commute = data.get("commute", "")
        return pd.Series([transit, amenities, commute])
    except Exception as e:
        print(f"Error enriching row (index: {row.name}): {e}")
        return pd.Series(["", "", ""])

# Loop for batch processing, assuming chunks of 50 rows
for i in range(0, 3):
    start_idx = i * 50
    end_idx = (i + 1) * 50

    df_chunk = df_sample.iloc[start_idx:min(end_idx, len(df_sample))].copy()

    if df_chunk.empty:
        print(f"No more rows to process; reached end of df_sample at iteration {i}.")
        break

    # Create new columns for the enriched data
    result = df_chunk.apply(enrich_row_safe, axis=1)
    df_chunk[["transit", "amenities", "commute"]] = result

    # Sleep for 5 seconds after processing each chunk to avoid rate limits
    time.sleep(5)

    collected_dfs.append(df_chunk)
    print(f"Enriched {len(df_chunk)} rows in chunk {i+1}. Total enriched so far: {sum(len(d) for d in collected_dfs)} rows.")

if collected_dfs:
    enriched_df_neighborhood = pd.concat(collected_dfs).reset_index(drop=True)
    print(f"\nSuccessfully enriched a total of {len(enriched_df_neighborhood)} rows.")
else:
    enriched_df_neighborhood = pd.DataFrame() # Create an empty DataFrame if no data was processed
    print("\nNo rows were successfully enriched.")
collected_dfs = []


Enriched 50 rows in chunk 1. Total enriched so far: 50 rows.
Enriched 50 rows in chunk 2. Total enriched so far: 100 rows.
Enriched 50 rows in chunk 3. Total enriched so far: 150 rows.

Successfully enriched a total of 150 rows.


KeyboardInterrupt: 

In [356]:
enriched_df_neighborhood.head()

,Unnamed: 0,sale_id,pinx,sale_date,sale_price,sale_nbr,sale_warning,join_status,join_year,latitude,...,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other,submarket,raw_text,transit,amenities,commute
0,1,1999..144,..2734100475,1999-01-05,150000,1,,demo,1999,47.544359,...,0,0,0,0,0,E,bedrooms=3; bathrooms=1; city=SEATTLE; transit...,"10 min walk to bus stop, 20 min light rail ride","5 min walk to coffee shops and restaurants, 15...",25 min to downtown Seattle
1,4,1999..660,..6145600690,1999-01-08,164000,1,,demo,1999,47.703824,...,0,0,0,0,0,B,bedrooms=2; bathrooms=1; city=SEATTLE; transit...,,,
2,6,1999..1155,..3438501120,1999-01-19,175000,1,,demo,1999,47.549525,...,0,0,0,0,0,F,bedrooms=4; bathrooms=1; city=SEATTLE; transit...,"10 min walk to bus stop, 20 min light rail ride","5 min walk to coffee shops, 10 min walk to gro...",25 min to downtown Seattle
3,9,1999..1354,..3336001680,1999-01-14,160000,1,,demo,1999,47.526324,...,0,0,0,0,0,E,bedrooms=3; bathrooms=1; city=SEATTLE; transit...,"10 min walk to bus stops, 20 min light rail ride","5 min walk to cafes and grocery stores, 15 min...",25 min to downtown Seattle
4,10,1999..1424,..7857000705,1999-01-19,135000,1,,demo,1999,47.550687,...,0,0,0,0,0,E,bedrooms=3; bathrooms=1; city=SEATTLE; transit...,,,


In [346]:
RAW_COLUMNS = ['bedrooms', 'bathrooms', 'city', 'transit', 'amenities', 'commute']
available_cols = [c for c in RAW_COLUMNS if c in enriched_df_neighborhood.columns]
print(len(available_cols))
assert available_cols, f"None of the expected raw columns found. Have: {enriched_df_neighborhood.columns.tolist()}"

def row_to_raw_text(r):
    parts = []
    for col in available_cols:
        val = r[col]
        parts.append(f"{col}={val}")
    return "; ".join(parts)

enriched_df_neighborhood["raw_text"] = enriched_df_neighborhood.apply(row_to_raw_text, axis=1)

# Embed and index raw text
raw_embeddings_enriched_neighborhood = np.array(
    emb_model.encode(
        enriched_df_neighborhood["raw_text"].tolist(),
        convert_to_numpy=True,
        show_progress_bar=True
    ),
    dtype="float32"
)
index_enriched_neighborhood = faiss.IndexFlatL2(raw_embeddings_enriched_neighborhood.shape[1])
index_enriched_neighborhood.add(raw_embeddings_enriched_neighborhood)

6


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [364]:
USER_QUERY = "Find me 2 bedroom 1 bathroom homes with walk to bus stops"

top_idx, top_dist = search_raw(USER_QUERY, k=100, index = index_enriched_neighborhood)
enriched_df_neighborhood.iloc[top_idx][available_cols + ["raw_text"]]

,bedrooms,bathrooms,city,transit,amenities,commute,raw_text
30,2,0,SEATTLE,,,,bedrooms=2; bathrooms=0; city=SEATTLE; transit...
96,2,0,SEATTLE,,,,bedrooms=2; bathrooms=0; city=SEATTLE; transit...
1,2,1,SEATTLE,,,,bedrooms=2; bathrooms=1; city=SEATTLE; transit...
7,2,1,SEATTLE,,,,bedrooms=2; bathrooms=1; city=SEATTLE; transit...
59,2,1,SEATTLE,,,,bedrooms=2; bathrooms=1; city=SEATTLE; transit...
...,...,...,...,...,...,...,...
11,3,1,SEATTLE,"10 min walk to bus stops, 20 min light rail ride","5 min walk to restaurants and shops, 15 min dr...",25 min to downtown Seattle,bedrooms=3; bathrooms=1; city=SEATTLE; transit...
130,1,1,SEATTLE,"10 min walk to bus stops, 20 min light rail ride","5 min walk to cafes and restaurants, 15 min dr...",25 min to downtown Seattle,bedrooms=1; bathrooms=1; city=SEATTLE; transit...
24,3,3,SEATTLE,"15 min walk to bus stop, 30 min to light rail","5 min drive to grocery stores, 10 min walk to ...",25 min to downtown Seattle,bedrooms=3; bathrooms=3; city=SEATTLE; transit...
55,4,1,SEATTLE,"10 min walk to bus stop, 20 min light rail ride","5 min walk to cafes and grocery stores, 15 min...",25 min to downtown Seattle,bedrooms=4; bathrooms=1; city=SEATTLE; transit...



---
<END>


We hope you had some fun learning how to build a retrieval engine and applying different prompt engineering techniques. Now, we will jump into learning how to scale these pipelines and cover some advanced topics.